In [ ]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as wtoken
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import (Activation, Dropout, Dense, Embedding, Flatten, BatchNormalization, GlobalMaxPooling1D, Lambda, concatenate, LSTM, Concatenate)
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os, string, gc, re

from tqdm import tqdm


import tldextract as tld


from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

# In addition to Tensorflow, pandas and sklearn, this notebook requires the following NLP/Spacy packages.
- conda install -c anaconda nltk
- conda install -c conda-forge tldextract
- conda install -c conda-forge wordcloud
- conda install -c conda-forge rake_nltk
- conda install -c conda-forge spacy
- !python3 -m spacy download en_core_web_lg
- !python3 -m spacy download en_core_web_md
- !python3 -m spacy download en_core_web_sm

In [ ]:
# A quick verification to see if the gpu drivers are ready to serve traning DL
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
features = ['keyword', 'location', 'text']

In [ ]:
path = '/media/moh/abunayla/Disaster_Tweets/Notebooks/data/'

In [ ]:
train = pd.read_csv(path + 'train.csv', usecols = features)

test = pd.read_csv(path + 'test.csv', usecols = features)

samsub = pd.read_csv(path +'sample_submission.csv')

Y =  pd.read_csv(path +'train.csv', usecols = ['target'])

In [ ]:
train.head()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
samsub.isna().sum()

In [ ]:
train.location.value_counts()

In [ ]:
Y.target.value_counts()

In [ ]:
test.location.value_counts()

In [ ]:
train.keyword.isna().sum()

In [ ]:
def extract_keywords(text, start_with_char='#'):
    """
    Take in string.
    Extracts words start with certain chars.
    Removes punctiouations
    Returns extracted words that are longer than 3 chars.
    """
    words=[]
    words.append([word.translate(str.maketrans('', '', string.punctuation)) for word in text.split() if word.startswith(start_with_char) and len(word)>3 ])
    
    return words

In [ ]:
extract_keywords('this is a #Test, #ER if it works then It is a #Great solution.')

In [ ]:
'airplane%20accident'.translate(str.maketrans('', '', string.punctuation))

In [ ]:
train.iloc[161]

In [ ]:
'airplane%20accident'.replace('%20', ' ')

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
syns = wn.synsets('blaze')

In [ ]:
syns

In [ ]:
synonyms =[]
antonyms =[]
for syn in wn.synsets('accident'):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

In [ ]:
synonyms[0].

In [ ]:
w1 = wn.synset('fire.n.01')
w2 = wn.synset('hazard.n.01')
print(w1.wup_similarity(w2))

In [ ]:
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C

In [ ]:
# import these modules 
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
  
print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora",pos ="a")) 
  
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a")) 

print("ablaze :", lemmatizer.lemmatize("ablaze")) 


In [ ]:
word = 'better'
print(lemmatizer.lemmatize(word)) 
print(lemmatizer.lemmatize(word, pos = 'a')) 

In [ ]:
# import TweetTokenizer() method from nltk 
from nltk.tokenize import TweetTokenizer 
  
# Create a reference variable for Class TweetTokenizer 
tk = TweetTokenizer() 
  
# Create a string input 
gfg = "Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J"
  
# Use tokenize method 
geek = tk.tokenize(gfg) 
  
print(geek)

In [ ]:
# import these modules 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer() 
  
# choose some words to be stemmed 
words = ["program", "programs", "programer", "programing", "programers"] 
words = ["fire", "firing", "on_fire", "inflamable", "ablaze"] 
  
for w in words: 
    print(w, " : ", ps.stem(w))

In [ ]:
word = 'ablaze'
print(word, " : ", ps.stem(word))

In [ ]:
print(wn.synset('fire.n.02').lemma_names())

In [ ]:
print(wn.synsets('fire'))

In [ ]:
#train.keyword.fillna('#MISSING', inplace = True)

In [ ]:
#train.keyword.isna().sum()

In [ ]:
#train.head()

### Convert all words to lower, this helps when using NLTK functions since they are use lower case words, e.g. the stopwords list.

In [ ]:
from rake_nltk import Rake

r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.

r.extract_keywords_from_text('Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding')


r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.

In [ ]:
text = 'on the outside you are ablaze and alive but you are dead inside'

In [ ]:
stop_words = stopwords.words('english')
word_tokens = word_tokenize(text) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w)
#print(word_tokens) 
print(filtered_sentence) 

In [ ]:
r.extract_keywords_from_text(' '.join(filtered_sentence))


r.get_ranked_phrases() 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
corpus = ['Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham\'s Wholesale Market http://t.co/irWqCEZWEU']

In [ ]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(corpus)

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
wn.synset('running.n.01').lowest_common_hypernyms(wn.synset('accident.n.01'))

Checking missing keyword against text if it has a keyword to fill the missing one in keyword column

In [ ]:
train.keyword.equals('USA')

In [ ]:
maybe I shoud use group by?

In [ ]:
maybe I shoud use group by

In [ ]:
unsupervised learning for text summarization

In [ ]:
part-of-speech tagging

In [ ]:
from nltk import word_tokenize as w_token
text = w_token('Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham\'s Wholesale Market http://t.co/irWqCEZWEU')

In [ ]:
pos_tag = nltk.pos_tag(text)
pos_tag

In [ ]:
	
text = nltk.Text(word.lower() for word in nltk.corpus.brown.words())
text.similar('woman')

In [ ]:
text.similar('bought')

In [ ]:
!pip install geocoder

In [ ]:
!pip uninstall geocoder

In [ ]:
import nltk

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        #if t.label() == 'NE':
        #entity_names.append(' '.join([child[0] for child in t]))
        #else:
        for child in t:
            entity_names.extend(extract_entity_names(child))

    return entity_names

In [ ]:
msg = ['USA', 'London', 'Kuala Lumpur', 'Water', 'Car']

In [ ]:
msg = ['I will attend GDC my ass in china, wuhan city next year, then I will go to tanzania.']

In [ ]:
for line in msg:
        sentences = nltk.sent_tokenize(line)
        tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
        tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
        chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

        entities = []
        for tree in chunked_sentences:
            entities.extend(extract_entity_names(tree))

        print(entities)

In [ ]:
import pycountry
text = 'I will attend GDC my ass in china, Mohanad wuhan city next year, then I will go to Tanzania, iraq.'
for country in pycountry.countries:
    if country.name.lower() in text:
        print(country.name)

In [ ]:
import pycountry
text = 'I will attend GDC my ass in China, London wuhan city next year, then I will go to tanzania.'
for country in pycountry.countries:
    print(country.name)

In [13]:
import spacy

nlp = spacy.load('en_core_web_lg')

doc = nlp('I will attend GDC my ass in China, London wuhan next year, baghad then I will go to tanzania. Maybe I will buy a Fool Phone and a car, and will eat Fish and Apple')

for ent in doc.ents:
    print(ent.text, ent.label_)

China GPE
London GPE
wuhan GPE
next year DATE
tanzania GPE
a Fool Phone PERSON
Fish WORK_OF_ART
Apple ORG


In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp('I will attend GDC my ass in China, London wuhan city next year, then I will go to tanzania. Maybe I will buy a phone and a car')

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

China 28 33 GPE
London 35 41 GPE
next year 53 62 DATE
tanzania 82 90 GPE


In [7]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("San Francisco considers banning sidewalk delivery robots")

# document level
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)

# token level
ent_san = [doc[0].text, doc[0].ent_iob_, doc[0].ent_type_]
ent_francisco = [doc[1].text, doc[1].ent_iob_, doc[1].ent_type_]
print(ent_san)  # ['San', 'B', 'GPE']
print(ent_francisco)  # ['Francisco', 'I', 'GPE']

[('San Francisco', 0, 13, 'GPE')]
['San', 'B', 'GPE']
['Francisco', 'I', 'GPE']


In [17]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp('I will attend GDC my ass in China, London wuhan city next year, then I will go to tanzania. Maybe I will buy a phone and a car')

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop, '\n')

I -PRON- PRON PRP nsubj X True True 

will will VERB MD aux xxxx True True 

attend attend VERB VB ccomp xxxx True False 

GDC GDC PROPN NNP dobj XXX True False 

my -PRON- DET PRP$ poss xx True True 

ass ass NOUN NN dobj xxx True False 

in in ADP IN prep xx True True 

China China PROPN NNP pobj Xxxxx True False 

, , PUNCT , punct , False False 

London London PROPN NNP compound Xxxxx True False 

wuhan wuhan PROPN NNP compound xxxx True False 

city city NOUN NN npadvmod xxxx True False 

next next ADJ JJ amod xxxx True True 

year year NOUN NN npadvmod xxxx True False 

, , PUNCT , punct , False False 

then then ADV RB advmod xxxx True True 

I -PRON- PRON PRP nsubj X True True 

will will VERB MD aux xxxx True True 

go go VERB VB ROOT xx True True 

to to ADP IN prep xx True True 

tanzania tanzania PROPN NNP pobj xxxx True False 

. . PUNCT . punct . False False 

Maybe maybe ADV RB advmod Xxxxx True False 

I -PRON- PRON PRP nsubj X True True 

will will VERB MD aux xxxx Tru

In [20]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.")
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

@aria_ahrary @aria_ahrary ROOT @aria_ahrary
control wild fires fires pobj of
California California pobj in
the Northern part part pobj in
the state state pobj of


In [22]:
import spacy
from spacy.symbols import nsubj, VERB

nlp = spacy.load("en_core_web_sm")
doc = nlp("@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.")

# Finding a verb with a subject from below — good
verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
print(verbs)

set()


In [ ]:
import spacy

nlp = spacy.load("de_core_news_sm")
doc = nlp("schöne rote Äpfel auf dem Baum")
print([token.text for token in doc[2].lefts])  # ['schöne', 'rote']
print([token.text for token in doc[2].rights])  # ['auf']

In [24]:
!python3 -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.1 MB/s eta 0:00:01     |███████████████████████▍        | 70.4 MB 874 kB/s eta 0:00:30
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=aec0ac6e41d065b7eee4a637de2343996fbd3d1ff4d9af8ea3fbcf2fbeecf7fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-yz83r_3f/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
